# LTL-f BASED-TRACE ALIGNMENT

From constraint to LTL-f: http://www.diag.uniroma1.it/degiacom/papers/2014/AAAI14.pdf

From LTL-f to DFA: http://ltlf2dfa.diag.uniroma1.it/

From LTL-f to automaton: https://github.com/whitemech/logaut

LTL2DFA library: https://github.com/whitemech/LTLf2DFA/

In [1]:
import re
from typing import Match, cast #, Tuple

from ltlf2dfa.parser.ltlf import LTLfParser
from logaut.backends.common.process_mona_output import (
    parse_automaton,
    parse_mona_output,
)

'''from logaut import ltl2dfa
from pylogics.parsers import parse_ltl'''

'from logaut import ltl2dfa\nfrom pylogics.parsers import parse_ltl'

## Constraint automaton

**CONSTRAINTS**

- [x] Chain precedence activity 16 - 17
- [x] Existence activity 1
- [x] Precedence activity 9 -10
- [x] Responded existence activity 5 - 6
- [x] Chain response activity 14 - 15
- [x] Not co-existence activity 19 -20
- [x] Not succession activity 20 -21
- [x] Not chain succession activity 22 - 23
- [x] Response activity 11 - 12
- [x] Absence2 activity 2


In [2]:
def postprocess_output(output: str) -> str:
    """
    Post-process MONA output.
    Capture the output related to the MONA DFA transitions.
    :param: the raw output from the LTLf2DFA tool.
    :return: the output associated to the DFA.
    """
    regex = re.compile(
        r".*(?=\nFormula is (valid|unsatisfiable)|A counter-example)",
        flags=re.MULTILINE | re.DOTALL,
    )
    match = regex.search(output)
    if match is None:
        raise Exception("cannot find automaton description in MONA output.")
    return cast(Match, regex.search(output)).group(0)

In [3]:
parser = LTLfParser()

constraint_formulas = {"existence":"F(a)", 
                       "absence2":"!(F((b & X(F(b)))))", 
                       "response":"G(k -> F(l))", 
                       "precedence":"((!(j) U i) | G(!(j)))",
                       "chain_response":"G((n -> X(o)))", 
                       "chain_precedence": "(!(q) & G((X(q) -> p)))",
                       "responded_existence":"(F(e) -> F(f))", 
                       "not_coexistence":"!((F(s) & F(t)))", 
                       "not_succession":"G((t -> !(F(u))))", 
                       "not_chain_succession":"G((v <-> !(X(w))))",
                       "formula_inventata":"(F(a) & F(b)) -> G(c)"}

In [27]:
#constraint_formulas = {"existence":"F(a)","response":"G(k -> F(l))"}
index = 1

all_automata = {}

for type_constr,f in constraint_formulas.items():
    build_automaton = {}
    build_automaton["name"] = type_constr
    build_automaton["formula"] = f

    all_states_constr = []
    final_states_constr = []
    init_states_constr = []
    rejecting_states_constr = []
    rho_constr_basic = []
    rho_tobe_negated = []

    ## Parser + dfa #####################################################
    formula = parser(f)       # returns an LTLfFormula
    dfa = formula.to_dfa(mona_dfa_out=True)
    mona_output = parse_mona_output(postprocess_output(dfa))
    automaton = parse_automaton(mona_output)
    #automaton.to_graphviz().render("automata/"+type_constr+".dfa", view=True)

    ## save all what is needed ##########################################
    init_states_constr = 's1'
    
    states = set.union(mona_output.rejecting_states, mona_output.accepting_states)
    states.remove(0)
    states = list(states)
    for elem in states:
        all_states_constr.append('s'+str(elem))

    for elem in list(mona_output.accepting_states):
        final_states_constr.append('s'+str(elem))

    alphabet = mona_output.variable_names

    for key,elem in mona_output.transitions.items():
        s_start = key
        if(s_start != 0):
            for k,e in elem.items():
                s_end = k
                sum = 0
                if(s_end != s_start):
                    comb = list(e)[0]
                    for char in comb:
                        if(char!='X'):
                            sum += int(char)
                    if(sum > 1):
                        continue
                    elif(sum == 1):
                        index1 = comb.find('1')
                        rho_constr_basic.append("s"+str(s_start)+" "+alphabet[index1]+" s"+str(s_end))
                    elif(sum == 0):
                        ## SBAGLIATO !!!!!!!!!!! #########################################
                        # vedi output di cella dove gestiamo le trans negate con l'alfabeto di
                        # traccia + alf di constr (ci sono trans ripetute)
                        indeces0 = [i for i in range(len(comb)) if comb[i] in '0']
                        res_list = [alphabet[i] for i in indeces0]
                        rho_tobe_negated.append([s_start,s_end,res_list])


    build_automaton["all_states"] = all_states_constr
    build_automaton["final_states"] = final_states_constr
    build_automaton["init_state"] = init_states_constr
    build_automaton["transitions"] = rho_constr_basic
    build_automaton["symbols_constr"] = alphabet
    build_automaton["negated_transitions"] = rho_tobe_negated

    all_automata["a"+str(index)] = build_automaton

    index += 1

In [19]:
for i in range(11):
    print(all_automata["a"+str(i+1)]["negated_transitions"])

[]
[]
[]
[]
[]
[[2, 1, ['P']]]
[]
[]
[]
[[1, 2, ['V']], [3, 2, ['V', 'W']]]
[[1, 2, ['A', 'B', 'C']], [4, 3, ['A', 'C']], [6, 5, ['B', 'C']], [8, 7, ['C']]]


## Load xes file
containing the trace

In [20]:
log_path = "dataset/logs/synthetic-logs/10constraints/1-constraint-inverted/log-from-10constr-model-1constr_inverted-1-50.xes"

In [21]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
voc = {}
for i in range(len(alphabet)):
    voc[i+1] = alphabet[i]

def convertNumberToChar (val):
    return voc[val]

In [22]:
# Legge dal file .xes, estrapola le tracce, trasforma il valore delle tracce in caratteri,
# infine aggiungile al log sottoforma di stringhe
def readLog (log_path):
    # Inizializzazione variabili
    flag = False;           # Indica quando dobbiamo leggere un evento dal file
    trace = [];             # Lista di eventi sottoforma di interi
    traceChar = [];         # Lista di eventi sottoforma di char
    traceString = "";       # Stringa composta da eventi sottoforma di char
    log = []               # Lista di tracce ognuna delle quali è una stringa di char

    # Apriamo il file e leggiamolo riga per riga
    f = open  (log_path)
    f1 = f.readlines()
    
    # Per ogni riga del file...
    for x in f1:
        # Se c'è un evento, attiviamo la flag
        if (x.__contains__("<event>")):
            flag = True
        # Se flag attiva e siamo sulla riga dove è presente il nome dell'evento,
        # estrapoliamo il nome dell'evento e appendiamolo a trace
        if (flag and x.__contains__('<string key="concept:name"')):
            val = x.split('value="activity ',1)[1]
            val = val.split('"')[0]                
            trace.append(val)
            flag = False
        # Quando non ci sono più eventi possiamo lavorare sulla traccia in questione
        if (x.__contains__("</trace>")):
            for event in trace:
                traceChar.append(convertNumberToChar(int(event)))  # Converti gli eventi in char 
            traceString = "".join(traceChar)                       # Lista di eventi -> stringa
            log.append(traceString)                                # Appendi stringa a log

            # Inizializza nuovamente le variabili
            trace = []
            traceChar = []
            traceString = ""
    return log

In [23]:
log = readLog(log_path) # list of traces

#trace = log[2]
#trace = 'stkai'
#print (trace)

In [29]:
#traces_log = log
traces_log = ['STAI']

all_traces = []

for t in traces_log:
    trace = {}
    
    symbols_trace = list(set(t))
    
    ## Build trace automaton ###################################
    rho_trace_basic = []
    Q_trace = []
    for i in range(len(trace)):
        Q_trace.append('t'+str(i))
        rho_trace_basic.append('t'+str(i)+" "+trace[i] +" "+ 't'+str(i+1))

    Q_trace.append('t'+str(len(trace)))
    init_state_trace = Q_trace[0]
    final_state_trace = Q_trace[-1]

    trace["symbols_trace"] = symbols_trace
    trace["init_state_trace"] = init_state_trace
    trace["final_state_trace"] = final_state_trace
    trace["Q_trace"] = Q_trace
    trace["rho_trace_basic"] = rho_trace_basic

    all_traces.append(trace)

    ## Now we add the transitions for the negated symbols ######

    for ID,automaton in all_automata.items():
        symb_constr = set(automaton["symbols_constr"])
        symb_trace = set(symbols_trace)
        all_symbs = list(set.union(symb_constr, symb_trace))
        print(all_symbs)

        trans1 = automaton["transitions"]
        trans = []
        for t_neg in automaton["negated_transitions"]:
            symbs = all_symbs.copy()
            print(t_neg[2])
            for t in t_neg[2]:
                symbs.remove(t)
            print("all_symbs after remove: ", symbs)
            for elem in symbs:
                trans.append("s"+str(t_neg[0])+" "+elem+" s"+str(t_neg[1]))
            print(trans)
            print("-----------")

        automaton["transitions"] = trans # + trans1
    
    print("___________________________")

['A', 'I', 'T', 'S']
['I', 'A', 'B', 'T', 'S']
['K', 'L', 'A', 'I', 'T', 'S']
['J', 'A', 'I', 'T', 'S']
['O', 'N', 'A', 'I', 'T', 'S']
['Q', 'A', 'I', 'T', 'S', 'P']
['P']
all_symbs after remove:  ['Q', 'A', 'I', 'T', 'S']
['s2 Q s1', 's2 A s1', 's2 I s1', 's2 T s1', 's2 S s1']
-----------
['A', 'F', 'I', 'T', 'S', 'E']
['A', 'I', 'T', 'S']
['I', 'A', 'U', 'T', 'S']
['T', 'A', 'I', 'W', 'V', 'S']
['V']
all_symbs after remove:  ['T', 'A', 'I', 'W', 'S']
['s1 T s2', 's1 A s2', 's1 I s2', 's1 W s2', 's1 S s2']
-----------
['V', 'W']
all_symbs after remove:  ['T', 'A', 'I', 'S']
['s1 T s2', 's1 A s2', 's1 I s2', 's1 W s2', 's1 S s2', 's3 T s2', 's3 A s2', 's3 I s2', 's3 S s2']
-----------
['I', 'A', 'C', 'B', 'T', 'S']
['A', 'B', 'C']
all_symbs after remove:  ['I', 'T', 'S']
['s1 I s2', 's1 T s2', 's1 S s2']
-----------
['A', 'C']
all_symbs after remove:  ['I', 'B', 'T', 'S']
['s1 I s2', 's1 T s2', 's1 S s2', 's4 I s3', 's4 B s3', 's4 T s3', 's4 S s3']
-----------
['B', 'C']
all_symbs afte

## Trace automaton

In [ ]:
rho_trace_basic = []           # rho = [[q0, w, q1],[q1,k,q2],...,[..,.,qn-1]]
Q_trace = []
for i in range(len(trace)):
    Q_trace.append('t'+str(i))
    rho_trace_basic.append('t'+str(i)+" "+trace[i] +" "+ 't'+str(i+1))

Q_trace.append('t'+str(len(trace)))
init_state_trace = Q_trace[0]
final_state_trace = Q_trace[-1]

print(Q_trace)
print(rho_trace_basic)

## Planning domain D

In [ ]:
domain_name = "domain_multi"
problem_name = "problem_multi"

In [ ]:
pddl_domain_initial = "(define (domain "+domain_name+") "\
                "(:requirements :strips :typing :action-costs) "\
                "(:types trace_state automaton_state - state activity) "

pddl_domain_predicates = "(:predicates (trace ?t1 - trace_state "\
                            "?e - activity "\
                            "?t2 - trace_state) "\
                            "(automaton ?s1 - automaton_state "\
                            "?e - activity "\
                            "?s2 - automaton_state) "\
                            "(cur_state ?s - state) "\
                            "(final_state ?s - state)) "


pddl_domain_actions =  "(:action sync "\
                        ":parameters (?t1 - trace_state ?e - activity ?t2 - trace_state) "\
                        ":precondition (and (cur_state ?t1) (trace ?t1 ?e ?t2)) "\
                        ":effect(and (not (cur_state ?t1)) (cur_state ?t2) "\
                        "(forall (?s1 ?s2 - automaton_state) "\
                        "(when (and (cur_state ?s1) "\
                        "(automaton ?s1 ?e ?s2)) "\
                        "(and (not (cur_state ?s1)) "\
                        "(cur_state ?s2)))))) "\
                        "(:action add "\
                        ":parameters (?e - activity) "\
                        ":effect (and (increase (total-cost) 1) "\
                        "(forall (?s1 ?s2 - automaton_state) "\
                        "(when (and (cur_state ?s1) "\
                        "(automaton ?s1 ?e ?s2)) "\
                        "(and (not (cur_state ?s1)) "\
                        "(cur_state ?s2)))))) "\
                        "(:action del "\
                        ":parameters (?t1 - trace_state ?e - activity "\
                        "?t2 - trace_state) "\
                        ":precondition (and (cur_state ?t1) (trace ?t1 ?e ?t2)) "\
                        ":effect (and (increase (total-cost) 1) "\
                        "(not (cur_state ?t1)) (cur_state ?t2))))"

pddl_domain = pddl_domain_initial + pddl_domain_predicates + pddl_domain_actions

print (pddl_domain)

## PDDL problem

In [ ]:
symbols = symbols_constr.copy()
'''print (Q_trace)
print (all_states_constr)
print (symbols)
print (rho_trace_basic)
print (rho_constr_basic)'''

In [ ]:
#type_constraints = ['existence', 'absence2', 'response', 'precedence', 'chain_response','responded_existence', 'chain_precedence', 'not_coexistence', 'not_succession', 'not_chain_succession']
type_constraints = ["existence","response"]

pddl_problem_initial = "(define (problem "+problem_name+") (:domain "+domain_name+") "
pddl_problem_objects = "(:objects "

for q in Q_trace:
    pddl_problem_objects += q+" "
pddl_problem_objects += "- trace_state "

for type_constr in type_constraints:
    for q in all_states_constr[type_constr]:
        pddl_problem_objects += q+" "
    pddl_problem_objects += "- automaton_state "

for s in symbols:
    pddl_problem_objects += s+" "
pddl_problem_objects += "- activity"

pddl_problem_objects += ") "


pddl_problem_init = "(:init (= (total-cost) 0) (cur_state "+init_state_trace+") "
for trace in rho_trace_basic:
    pddl_problem_init += "(trace "+trace+") "
pddl_problem_init += "(final_state "+final_state_trace+") "

for type_constr in type_constraints:
    pddl_problem_init += "(cur_state "+init_states_constr[type_constr]+") " 
    for trace in rho_constr_basic[type_constr]:
        pddl_problem_init += "(automaton "+trace+") "

    for i in range(len(final_states_constr[type_constr])):
        pddl_problem_init += "(final_state "+final_states_constr[type_constr][i]+") "
pddl_problem_init += ") "


pddl_problem_goal = "(:goal (forall (?s - state) "\
                    "(imply (cur_state ?s) (final_state ?s)))) "
pddl_problem_metric = "(:metric minimize (total-cost)))"
pddl_problem = pddl_problem_initial + pddl_problem_objects + pddl_problem_init + pddl_problem_goal + pddl_problem_metric
print (pddl_problem)

## Save .pddl files

In [ ]:
file1 = open("PDDL/"+domain_name+".pddl", "w")
file1.write(pddl_domain)
file1.close()

file2 = open("PDDL/"+problem_name+".pddl", "w")
file2.write(pddl_problem)
file2.close()

Fast-downward planner:
`./fast-downward.py domain_multi.pddl problem_multi.pddl --search "astar(blind())"`